# Clasificación de Frutas Frescas vs. Podridas con ResNet18
## Notebook de Entrenamiento e Inferencia

Este notebook implementa un modelo de clasificación de imágenes para distinguir entre frutas frescas y podridas utilizando transfer learning con ResNet18.

## 1. Configuración Inicial

In [ ]:
# Importación de librerías
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from google.colab import drive
from PIL import Image

print("✅ Librerías importadas correctamente")

## 2. Descarga y Preparación del Dataset

In [ ]:
try:
    # Descarga del dataset desde Kaggle
    path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
    print("✅ Dataset descargado correctamente")
    print("Path to dataset files:", path)

    # Configuración básica
    batch_size = 128  # Tamaño reducido para evitar problemas de memoria
    img_size = 224    # Tamaño requerido por ResNet
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Dispositivo de ejecución: {device}")

    # Transformaciones para las imágenes
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    print("✅ Configuración y transformaciones definidas")

## 3. Carga de Datos y Preprocesamiento

In [ ]:
# Carga de datasets
train_data_dir = os.path.join(path, 'dataset', 'train')
val_data_dir = os.path.join(path, 'dataset', 'test')

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_data_dir, transform=transform)

# Información sobre los datos
class_names = train_dataset.classes
print(f"🔍 Clases detectadas: {class_names}")
print(f"📊 Total de imágenes de entrenamiento: {len(train_dataset)}")
print(f"📊 Total de imágenes de validación: {len(val_dataset)}")

# Creación de DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
print("✅ Dataset cargado y dividido correctamente")

## 4. Configuración del Modelo (Transfer Learning)

In [ ]:
# Carga de ResNet18 pre-entrenada
model = models.resnet18(pretrained=True)
print("🎯 ResNet18 cargado (pre-entrenado en ImageNet)")

# Congelación de parámetros
for param in model.parameters():
    param.requires_grad = False
print("❄️ Parámetros congelados (excepto última capa)")

# Modificación de la última capa
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))
print(f"🔄 Capa FC reemplazada para {len(class_names)} clases")

# Movimiento del modelo al dispositivo
model = model.to(device)
print(f"📌 Modelo movido a {device}")
print("✅ Modelo definido correctamente")

## 5. Configuración del Entrenamiento

In [ ]:
# Definición de función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
epochs = 10

print("⚙️ Configuración de entrenamiento:")
print(f"- Función de pérdida: {criterion.__class__.__name__}")
print(f"- Optimizador: {optimizer.__class__.__name__} (lr=0.001)")
print(f"- Épocas: {epochs}")
print("✅ Configuración de entrenamiento completada")

## 6. Proceso de Entrenamiento

In [ ]:
# Listas para métricas
train_losses = []
val_losses = []
val_accuracies = []

print("🚀 Comenzando entrenamiento...")

for epoch in range(epochs):
    # Fase de entrenamiento
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    
    # Fase de validación
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    accuracy = 100 * correct / total
    val_accuracies.append(accuracy)
    
    # Progreso
    print(f"✅ Epoch {epoch+1}/{epochs} - "
          f"Train Loss: {epoch_loss:.4f} - "
          f"Val Loss: {val_loss:.4f} - "
          f"Accuracy: {accuracy:.2f}%")

print("🎉 Entrenamiento completado!")

## 7. Visualización de Resultados

In [ ]:
# Gráficos de pérdida y precisión
plt.figure(figsize=(12, 5))

# Gráfico de pérdidas
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Entrenamiento')
plt.plot(val_losses, label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de Pérdida')
plt.legend()

# Gráfico de precisión
plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validación', color='green')
plt.xlabel('Épocas')
plt.ylabel('Precisión (%)')
plt.title('Precisión en Validación')
plt.legend()

plt.tight_layout()
plt.show()

## 8. Guardado del Modelo

In [ ]:
try:
    model_path = "modelo.pth"
    torch.save(model.state_dict(), model_path)
    print(f"\n💾 Modelo guardado como: {model_path}")
    print(f"Tamaño del archivo: {os.path.getsize(model_path)/1024:.2f} KB")
except Exception as e:
    print(f"❌ Error al guardar el modelo: {e}")

## 9. Fase de Inferencia

In [ ]:
print("\n--- Fase de Inferencia ---")
# Selección de imagen de ejemplo
sample_image_path, _ = val_dataset.samples[0]
print(f"⏳ Realizando inferencia en la imagen: {sample_image_path}")

# Preprocesamiento
image = Image.open(sample_image_path).convert('RGB')
image = transform(image).unsqueeze(0).to(device)

# Inferencia
model.eval()
with torch.no_grad():
    outputs = model(image)
    _, predicted_class_index = torch.max(outputs, 1)

# Resultado
predicted_class_name = class_names[predicted_class_index.item()]
print(f"✅ La fruta detectada es: {predicted_class_name}")

## 10. Manejo de Errores

In [ ]:
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")